# Numerical Methods

# Lecture 5: Numerical Linear Algebra I

In [2]:
import numpy as np
import scipy.linalg as sl

## Learning objectives:

* Manipulation of matrices and matrix equations in Python


* Reminder on properties of matrices (from MM1): determinants, singularity etc


* Algorithms for the solution of linear systems


* Gaussian elimination, including back substitution


Note that this is the first of 3 lectures on numerical linear algebra, which will take us to the end of this module.

## Introduction - Linear (matrix) systems

Recall from your Mathematical Methods I course that the we can re-write a system of simultaneous (linear) equations in matrix form.  

For example, in week 4 (I think) of MM1 you considered the following example (or one very similar):

\begin{eqnarray*}
  2x + 3y &=& 7 \\
   x - 4y &=& 3
\end{eqnarray*} 

and it was noted that this can be completely equivalently written in matrix form as 

$$
\left(
  \begin{array}{rr}
    2 & 3 \\
    1 & -4  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    7 \\
    3 \\
  \end{array}
\right)
$$

More generally, consider the arbitrary system of $n$ linear equations for $n$ unknowns

\begin{eqnarray*}
  A_{11}x_1 + A_{12}x_2 + \dots + A_{1n}x_n &=& b_1 \\ 
  A_{21}x_1 + A_{22}x_2 + \dots + A_{2n}x_n &=& b_2 \\ 
  \vdots &=& \vdots \\ 
  A_{n1}x_1 + A_{n2}x_2 + \dots + A_{nn}x_n &=& b_n
\end{eqnarray*}

where $A_{ij}$ are the constant coefficients of the linear system, $x_j$ are the unknown variables, and $b_i$ are the terms on the right hand side (RHS).  

Here the index $i$ is referring to the equation number (the row in the matrix below), with the index $j$ referring to the component of the unknown vector $\boldsymbol{x}$ (the column of the matrix).

In this module we will always assume that we are dealing with real numbers and so we can write that $A\in\mathbb{R}^{n\times n}$ or $A= \{A_{ij}\}_{n\times n}, \, A_{ij}\in \mathbb{R}$.

This system of equations can be represented as the matrix equation $A\boldsymbol{x}=\boldsymbol{b}$: 

$$
\left(
  \begin{array}{cccc}
    A_{11} & A_{12} & \dots & A_{1n} \\
    A_{21} & A_{22} & \dots & A_{2n} \\
    \vdots & \vdots & \ddots & \vdots \\
    A_{n1} & A_{n2} & \dots & A_{nn} \\
  \end{array}
\right)\left(
         \begin{array}{c}
           x_1 \\
           x_2 \\
           \vdots \\
           x_n \\
         \end{array}
       \right)  = \left(
                   \begin{array}{c}
                     b_1 \\
                     b_2 \\
                     \vdots \\
                     b_n \\
                   \end{array}
                 \right)
$$

We can easily solve the above $2 \times 2$ example of two equations and two unknowns using substitution (e.g. multiply the second equation by 2 and subtract the first equation from the resulting equation to eliminate $x$ and hence allowing us to find $y$, then we could compute $x$ from the first equation). 

Doing this we find:

$$ x=37/11, \quad y=1/11.$$

In MM1 you also considered $3 \times 3$ examples which were a little more complicated but still doable.  This lecture considers the case of $n \times n$ where $n$ could easily be billions! Cases this large can easily arise when you solve a differential equation numerically on a discrete mesh or grid. Here you would typically obtain one unknown and one (discrete, linear or nonlinear) equation at very grid point. You could generate an arbitrarily large matrix system simply by generating a finer mesh (imagine a mesh with 1000 grid points in each of the $x$, $y$ and $z$ directions!).

Cases where the matrix is non-square, i.e. of shape $m \times n$ where $m\ne n$ ($A\in\mathbb{R}^{m\times n}$) correspond to the (over- or under-determined) system where you have more or less equations than unknowns - we won't consider these in this lecture but for those of you that take the optional module *Geophysical Inversion* this will come up.

## Matrices in Python

We have already used NumPy arrays to store one-dimensional vectors of numbers.

The convention is that these are generally considered to be *column* vectors and have shape $n \times 1$.

We can extend to higher dimensions through the introduction of matrices as two-dimensional arrays (more generally, vectors and matrices are just two examples of tensors). 

We use subscript indices to identify each component of the array or matrix, i.e. we can identify each component of the vector $\boldsymbol{v}$ by $v_i$, and each component of the matrix $A$ by $A_{ij}$.  

Note that it is a convention that vectors are either underlined or bold, and generally lower case letters, whereas matrices are generally plain capital letters.

The *dimension* or *shape* of a vector/matrix is the number of rows and columns it possesses, i.e. $n \times 1$ and $m \times n$ for the examples above.

Here is an example of how we can extend our use of the NumPy array object to two dimensions in order to define a matrix $A$ and some examples of some operations we can make on it.

In [3]:
A = np.array([[10., 2., 1.],[6., 5., 4.],[1., 4., 7.]])

print(A)

[[10.  2.  1.]
 [ 6.  5.  4.]
 [ 1.  4.  7.]]


In [4]:
# the total size of the array storing A - here 9 for a 3x3 matrix
print(np.size(A))  

9


In [5]:
# the number of dimensions of the matrix A
print(np.ndim(A) )    

2


In [6]:
# the shape of the matrix A
print(np.shape(A))

(3, 3)


In [7]:
# and so if we need the number of rows say:
print('The number of rows in A is', np.shape(A)[0])

The number of rows in A is 3


In [8]:
# the transpose of the matrix A
print(A.T)

[[10.  6.  1.]
 [ 2.  5.  4.]
 [ 1.  4.  7.]]


In [9]:
# the inverse of the matrix A - computed using a scipy algorithm
print(sl.inv(A))

[[ 0.14285714 -0.07518797  0.02255639]
 [-0.28571429  0.51879699 -0.2556391 ]
 [ 0.14285714 -0.28571429  0.28571429]]


In [10]:
# the determinant of the matrix A - computed using a scipy algorithm
print(sl.det(A))

133.00000000000003


In [11]:
# Multiply A with its inverse using the @ matrix multiplication operator. 
# Note that due to roundoff errors the off diagonal values are not exactly zero.
print(A @ sl.inv(A))

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.22044605e-16  1.00000000e+00  2.22044605e-16]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]]


In [12]:
# same way to achieve the same thing
print(A.dot(sl.inv(A)))
# the @ operator is realtively new so you may still see use of dot in some code/courses

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.22044605e-16  1.00000000e+00  2.22044605e-16]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]]


In [13]:
# note that the * operator simply does operations element-wise - here this
# is not what we want!
print(A*sl.inv(A))
# if you're familiar with Matlab this poitwise operation is what you get with dot-star: ".*"

[[ 1.42857143 -0.15037594  0.02255639]
 [-1.71428571  2.59398496 -1.02255639]
 [ 0.14285714 -1.14285714  2.        ]]


In [14]:
# how to initialise a vector of zeros 
print(np.zeros(3))

[0. 0. 0.]


In [15]:
# how to initialise a matrix of zeros 
print(np.zeros((3,3)))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [16]:
# how to initialise a 3rd-order tensor of zeros
print(np.zeros((3,3,3)))

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [17]:
# shortcut to create an array of zero of the same size as A
B = np.zeros_like(A)
print(B)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [18]:
# how to initialise the identity matrix, I or Id
print(np.eye(3))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Using the inverse matrix to compute the solution to a linear system

Let's quickly consider the $2 \times 2$ case from MM1 recreated above where we claimed that $x=37/11$ and $y=1/11$.  

To solve the matrix equation 

$$ A\boldsymbol{x}=\boldsymbol{b} $$

we can simply multiply both sides by the inverse of the matrix $A$ (if $A$ is invertible and if we know what the inverse is of course!):

\begin{align}
A\boldsymbol{x} & = \boldsymbol{b}\\
\implies A^{-1}A\boldsymbol{x} & = A^{-1}\boldsymbol{b}\\
\implies I\boldsymbol{x} & = A^{-1}\boldsymbol{b}\\
\implies \boldsymbol{x} & = A^{-1}\boldsymbol{b}
\end{align}

so we can find the solution $\boldsymbol{x}$ by multiplying the inverse of $A$ with the RHS vector $\boldsymbol{b}$.

### <span style="color:blue">Exercise 5.1: Solving a linear system </span>
Formulate and solve the linear system and check you get the answer quoted above.  To check that the matrix has an inverse can you remember what you need the determinant of the matrix to be?

Note that `np.allclose` is a convenient function for checking if two arrays are the same (to round-off error).

In [19]:
import numpy as np
import scipy.linalg as sl

A = np.array([[2., 3.], [1., -4.]])
B = np.array([7, 3])

assert sl.det(A) != 0, "determinant of the matrix = 0 --> no inverse!!"

x = sl.inv(A) @ B

print(x)

[3.36363636 0.09090909]


## Slicing 

Hopefully from your year 1 Python module you recall that we can use *slicing* to extract entries from arrays or lists. 

We can do the same for matrices (2D numpy arrays) - recall that our numbering of entries starts from zero:

In [20]:
A=np.array([[10., 2., 1.],[6., 5., 4.],[1., 4., 7.]])
print(A)

[[10.  2.  1.]
 [ 6.  5.  4.]
 [ 1.  4.  7.]]


In [21]:
# extract a single entry, first row, second column
print(A[0,1])

2.0


In [22]:
# extract the first row
print(A[0,:])

[10.  2.  1.]


In [23]:
# the last row
print(A[-1,:])

[1. 4. 7.]


In [24]:
# the second column
print(A[:,1])

[2. 5. 4.]


In [25]:
# extract a 2x2 sub-matrix
print(A[1:3,1:3])

[[5. 4.]
 [4. 7.]]


In [26]:
# Replace the third column of A
A[:,2] = np.array([99.,98.,97.])
print(A)

[[10.  2. 99.]
 [ 6.  5. 98.]
 [ 1.  4. 97.]]


### <span style="color:blue">Exercise 5.2: Matrix manipulation in Python </span>

Let
$$
A = \left(
  \begin{array}{ccc}
    1 & 2 & 3 \\
    4 & 5 & 6 \\
    7 & 8 & 9 \\
  \end{array}
\right)
\mathrm{\quad\quad and \quad\quad}
b = \left(
  \begin{array}{c}
    2 \\
    4 \\
    6 \\
  \end{array}
\right)
$$


- Store $A$ and $b$ in NumPy array structures. Print them.


- Print their shape and size. What is the difference ?


- Create a NumPy array $I$ containing the identity matrix $I_3$. Perform the update $A = A+I$.


- Substitute the 3rd column of $A$ with $b$.


- Solve the linear system $Ax=b$ - what's your solution?

In [27]:
A = np.array([[1. , 2., 3.], [4., 5., 6.], [7., 8., 9.]])

" 1D --> only column"
b = np.array([2., 4., 6.])

print("Shape A B")
print(np.shape(A))
print(np.shape(b))

print("Size A B")
print(np.size(A))
print(np.size(b))

I = np.eye(3)

A = A + I

# Replace the third column of A with b
" every row but 3rd column"
A[:,2] = b
print(A)

print(sl.inv(A) @ b)

Shape A B
(3, 3)
(3,)
Size A B
9
3
[[2. 2. 2.]
 [4. 6. 4.]
 [7. 8. 6.]]
[ 0.00000000e+00 -1.11022302e-16  1.00000000e+00]


## Properties of matrices: determinants, singularity, solvability of linear systems, etc

Consider $N$ linear equations in $N$ unknowns, which we know from above can be written as $A\boldsymbol{x}=\boldsymbol{b}$. 

From MM1 you learnt that this system has a *unique solution* provided that the determinant of $A$, $\det(A)$, is non-zero. In this case the matrix is said to be *non-singular*.

If $\det(A)=0$ (with $A$ then termed a *singular matrix*), then the linear system does *not* have a unique solution, it may have either infinite *or* no solutions.

For example, consider

$$
\left(
  \begin{array}{rr}
    2 & 3 \\
    4 & 6  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    4 \\
    8 \\
  \end{array}
\right)
$$

The second equation is simply twice the first, and hence a solution to the first equation is also automatically a solution to the second equation. 

We hence only have one *linearly-independent* equation, and our problem is under-constrained: we effectively only have one eqution for two unknowns with infinitely many possibly solutions. 

If we replaced the RHS vector with $(4,7)^T$, then the two equations would be contradictory: in this case we have no solutions.

Note that a set of vectors where one can be written as a linear sum of the others are termed *linearly-dependent*. When this is not the case the vectors are termed *linearly-independent*.

The following properties of a square $n\times n$ matrix are equivalent:


* $\det(A)\ne 0$ - $A$ is non-singular


* The columns of $A$ are linearly independent


* The rows of $A$ are linearly independent


* The columns of A *span* $n$-dimensional space (recall MM1 - we can reach any point in $\mathbb{R}^N$ through a linear combination of these vectors - note that this is simply what the operation $A\boldsymbol{x}$ is doing of course if you write it out)


* $A$ is invertible, i.e. there exists a matrix $A^{-1}$ such that $A^{-1}A = A A^{-1}=I$


* The matrix system $A\boldsymbol{x}=\boldsymbol{b}$ has a unique solution for every RHS vector $b$


## Gaussian elimination - method


The Gaussian elimination algorithm is simply a systematic implementation of the method of equation substitution we used above to solve the $2\times 2$ system (i.e. where we "multiply the second equation by 2 and subtract the first equation from the resulting equation to *eliminate* $x$ and hence allowing us to find $y$, we can then compute $x$ from the first equation").

So *Gaussian elimination*  as the method is attributed to the mathematician *Gauss* (although it was certainly known before his time) and *elimination* as we seek to eliminate unknowns.

To perform this method for arbitrarily large systems (on paper) we form the so-called ***augmented matrix***

$$
[A|\boldsymbol{b}] = 
\left[
  \begin{array}{rr|r}
    2 & 3 & 7 \\
    1 & -4 & 3  \\
  \end{array}
\right]
$$

Note that this encodes the equations (including the RHS values); we can perform so-called *row operations* and as long as we are consistent with what we do for the LHS and RHS components then what this system is describing will not change - a solution to the updated system will be the same as the solution to the original system. 

Our task is to update the system so we can easily read off the solution - of course this is exactly what we do via the substitution approach:

First we multiplied the second equation by 2, this yield the updated augmented matrix:

$$
\left[
  \begin{array}{rr|r}
    2 & 3 & 7 \\
    2 & -8 & 6 \\
  \end{array}
\right]
$$

We can use the following notation to describe this operation:

$$ Eq. (2) \leftarrow 2\times Eq. (2) $$

Note importantly that this does not change anything about what these pair of equations are telling us about the unknown solution vector $\boldsymbol{x}$ which although it doesn't appear is implicitly defined by this augmented equation.

<br><br>
What I mean by this is that the linear systems encoded in the original augmented system and the updated one are completely equivalent in terms of their solutions:

$$
\left(
  \begin{array}{rr}
    2 & 3 \\
    1 & -4  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    7 \\
    3 \\
  \end{array}
\right)
\iff
\left(
  \begin{array}{rr}
    2 & 3 \\
    2 & -8  \\
  \end{array}
\right)\left(
  \begin{array}{c}
    x \\
    y \\
  \end{array}
\right) = \left(
  \begin{array}{c}
    7 \\
    6 \\
  \end{array}
\right)
$$

and this will be the case for any multiplication of a row, swapping or rows, or addition/subtraction of rows, as long as we remember to also apply the consistent operations on the RHS vector.

<br><br>

The next step was to subtract the first equation from the updated second ($ Eq. (2) \leftarrow Eq. (2) - Eq. (1) $):

$$
\left[
  \begin{array}{rr|r}
    2 & 3 & 7 \\
    0 & -11 & -1 \\
  \end{array}
\right]
$$

The square matrix that is now in the $A$ position of this augmented system is an example of an ***upper-triangular*** matrix - all entries below the diagonal are zero.  

For such a matrix we can perform ***back substitution*** - starting at the bottom to solve trivially for the final unknown ($y$ here which clearly takes the value $-1/-11$), and then using this knowledge working our way up to solve for each remaining unknown in turn, here just $x$ (solving $2x + 3\times (1/11) = 7$).

Note that we can perform the similar substitution if we had a lower triangular matrix, first finding the first unknown and then working our way forward through the remaining unknowns - hence in this case *forward substitution*.

Note that if we wished we could of course continue working on the augmented matrix to make the $A$ component diagonal: divide the second equation by 11 and multiply by 3 ($ Eq. (2) \leftarrow (3/11)\times Eq. (2) $) and add it to the first ($ Eq. (1) \leftarrow Eq. (1) +  Eq. (2) $):

$$
\left[
  \begin{array}{rr|r}
    2 & 0 & 7-3/11\\
    0 & -3 & -3/11 \\
  \end{array}
\right]
$$

and we can further make it the identity by dividing the rows by $2$ and $-3$ respectively ($ Eq. (1) \leftarrow (1/2)\times Eq. (1) $, $ Eq. (2) \leftarrow (-1/3)\times Eq. (2) $) :

$$
\left[
  \begin{array}{rr|r}
    1 & 0 & (7-3/11)/2 \\
    0 & 1 & 1/11 \\
  \end{array}
\right]
$$

Each of these augmented matrices encodes exactly the same information as the original matrix system in terms of the unknown vector $\boldsymbol{x}$, and hence this is telling us that

$$ \boldsymbol{x} = I \boldsymbol{x} = \left[
  \begin{array}{c}
    (7-3/11)/2 \\
    1/11 \\
  \end{array}
\right]
$$

i.e. exactly the solution we found when we performed back substitution from the upper-triangular form of the augmented system.


### <span style="color:blue">Exercise 5.3: Gaussian elimination $3 \times 3$ example (by hand) </span>

Consider the system of linear equations

\begin{align*}
  2x + 3y - 4z &= 5 \\
  6x + 8y + 2z &= 3 \\
  4x + 8y - 6z &= 19
\end{align*}

write this in matrix form, form the corresponding augmented system and perform row operations until you get to upper-triangular form, find the solution using back substitution (**do this all with pen and paper**).

Write some code to check your answer using `sl.inv(A) @ b`.

You should find $x=-6$, $y=5$, $z=-1/2$.


## Gaussian elimination - algorithm and code

As mentioned above, notice that we are free to perform the following operations on the augmented system without changing the corresponding solution:


* Exchange two rows (refer to the section on *partial pivoting* next week)


* Multiply a row by a non-zero constant ($Eq. (i)\leftarrow \lambda \times Eq.(i)$)


* Subtracting a (non-zero) multiple of one row with another ($Eq. (i)\leftarrow Eq. (i) - \lambda \times Eq.(j)$)



Note that the equation/row being subtracted (the one that is being used to cancel entries) is termed the *pivot*.


<br>

Let's consider the algorithm mid-way working on an arbitrary matrix system, i.e. assume that the first $k$ rows (i.e. above the horizontal dashed line in the matrix below) have already been transformed into upper-triangular form, while the equations/rows below are not yet in this form.

The augmented equation in this case can be assumed to look like

$$
\left[
  \begin{array}{rrrrrrrrr|r}
    A_{11} & A_{12} & A_{13} & \cdots & A_{1k} & \cdots & A_{1j} & \cdots & A_{1n} & b_1 \\
    0      & A_{22} & A_{23} & \cdots & A_{2k} & \cdots & A_{2j} & \cdots & A_{2n} & b_2 \\
    0      & 0      & A_{33} & \cdots & A_{3k} & \cdots & A_{3j} & \cdots & A_{3n} & b_3 \\
    \vdots & \vdots & \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots & \vdots \\
    0      & 0      & 0      & \cdots & A_{kk} & \cdots & A_{kj} & \cdots & A_{kn} & b_k \\    
\hdashline
    \vdots & \vdots & \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots & \vdots \\
    0      & 0      & 0      & \cdots & A_{ik} & \cdots & A_{ij} & \cdots & A_{in} & b_i \\
    \vdots & \vdots & \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots & \vdots \\
    0      & 0      & 0      & \cdots & A_{nk} & \cdots & A_{nj} & \cdots & A_{nn} & b_n \\
\end{array}
\right]
$$

<br>

Remember that here as we are mid-way through the algorithm the $A$'s and $b$'s in the above are **not** the same as in the original system! [this is something we will need to keep in mind when we code up our algorithm - are we happy for the input $A$ etc to be over-written?]

<br>

Assuming that we are at this stage in the algorithm, our aim as a next step in the algorithm is to use row $k$ (the "pivot row") to *eliminate* $A_{ik}$, and we need to do this for all of the rows $i$ below the pivot, i.e. for all $i>k$.

[Note that since we will be implementing some code based on a mathematical description of the algorithm, it's important that we get our head around these indices and their ranges - making sure we are clear what this is saying (and that it's correct) we significantly ease our implementation.]

<br>

Note that the zeros to the left of the leading term in the pivot row means that these operations will not mess up the fact that we have all the zeros we are looking for in the lower left part of the matrix. This is (a) what we want, but also (b) an observation that means we don't need to perform these operations in our implementation.

<br>

To eliminate $A_{ik}$ for a single row $i$ we need to perform the operation 
$$ Eq. (i)\leftarrow Eq. (i) - \frac{A_{ik}}{A_{kk}} \times Eq.(k) $$

or equivalently, mathematically

\begin{align}
A_{ij} &\leftarrow A_{ij} - \frac{A_{ik}}{A_{kk}} A_{kj}, \quad j=k,k+1,\ldots,n\\
b_i &\leftarrow b_i - \frac{A_{ik}}{A_{kk}} b_{k}
\end{align}

As commented above, $j$ only needs to run from $k$ upwards as we can assume that the earlier entries in column $i$ have already been set to zero, and also that the corresponding terms from the pivot row are also zero (we don't need to perform operations that we know involve the addition of zeros!).

<br>

Finally we note that in order to eliminate these entries for all rows below the pivot we need to repeat for all $i>k$.

And finally finally, note that all entries from the first down to the **second to last** will be used as pivot rows - we don't use the final row as there are no entries below it needing to be set to zero!

### <span style="color:blue">Exercise 5.3: Gaussian elimination</span>

Write some code that takes a matrix $A$ and a vector $\boldsymbol{b}$ and converts it into upper-triangular form using the above algorithm. 

For the $2 \times 2$ and $3\times 3$ examples from above compare the resulting $A$ and $\boldsymbol{b}$ you obtain following elimination.


In [28]:
def upper_triangle(A, b):
    """ A function to covert A into upper triangluar form through row operations.
    The same row operations are performed on the vector b.
    
    Note that this implementation does not use partial pivoting which is introduced below.
    
    Also note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """
    n = np.size(b)
    rows, cols = np.shape(A)
    "np.shape gives tuple"

    # check A is square
    assert(rows == cols)
    # and check A has the same numner of rows as the size of the vector b (columnar)
    assert(rows == n)

    # Loop over each pivot row - all but the last row which we will never need to use as a pivot
    " k = row/ column (square matrix)"
    for k in range(n-1):
        # Loop over each row below the pivot row, including the last row which we do need to update
        for i in range(k+1, n):
            "for EACH row below pivot row (i = row below pivot)"
            "loop for all rows below pivot to turn column k into 0"
            "k+1 --> after pivot row. start: k+1 end: n"

            # scaling factor for row
            "Row i (below pivot row) column k"
            "k works bc square matrix"
            s = (A[i, k] / A[k, k])

            # update current row A by looping over column j
            "start from k as we assume entries before this are already zero"
            '''for j in range(k, n):
                A[i, j] = A[i, j] - s * A[k, j]'''
            
            A[i, :] = A[i, :] - s * A[k, :]

            # update corresponding b
            b[i] = b[i] - s * b[k]


# Test our code on our 2x2 and 3x3 examples from above

# 2x2 example from above
A = np.array([[2., 3.], 
                [1., -4.]])
b = np.array([7., 3.])

upper_triangle(A, b)


print('\nOur A matrix following row operations to transform it into upper-triangular form:')
print(A)
print('The correspondingly updated b vector:')
print(b)

# 3x3 example from homework exercise
A = np.array([[2., 3., -4.],
              [6., 8., 2.],
              [4., 8., -6.]])
b = np.array([5., 3., 19.])

upper_triangle(A, b)


print('\nOur A matrix following row operations to transform it into upper-triangular form:')
print(A)
print('The correspondingly updated b vector:')
print(b)



Our A matrix following row operations to transform it into upper-triangular form:
[[ 2.   3. ]
 [ 0.  -5.5]]
The correspondingly updated b vector:
[ 7.  -0.5]

Our A matrix following row operations to transform it into upper-triangular form:
[[ 2.  3. -4.]
 [ 0. -1. 14.]
 [ 0.  0. 30.]]
The correspondingly updated b vector:
[  5. -12. -15.]


### Back substitution

Now we have a function that transforms our augmented system into the upper-triangular form

$$
\left[
  \begin{array}{rrrrr|r}
    A_{11} & A_{12} & A_{13} & \cdots & A_{1n} &  b_1 \\
    0      & A_{22} & A_{23} & \cdots & A_{2n} &  b_2 \\
    0      & 0      & A_{33} & \cdots & A_{3n} &  b_3 \\
    \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
    0      & 0      & 0      & \cdots & A_{nn} &  b_n \\    
\end{array}
\right]
$$

where the solution $\boldsymbol{x}$ of the original system also satisfies $A\boldsymbol{x}=\boldsymbol{b}$ for the $A$ and $\boldsymbol{b}$ in the above upper-triangular form (rather than the original $A$ and $\boldsymbol{b}$!).

What's our next step to obtain the solution vector $\boldsymbol{x}$?

<br>

We can solve the final equation row to yield

$$x_n = \frac{b_n}{A_{nn}}$$

The second to last equation then yields (note we've introduced a comma in the subscripts here simply to make the more complex double indices easier to read)

\begin{align}
A_{n-1,n-1}x_{n-1} + A_{n-1,n}x_n &= b_{n-1}\\
\implies x_{n-1} = \frac{b_{n-1} - A_{n-1,n}x_n}{A_{n-1,n-1}}\\
\implies x_{n-1} = \frac{b_{n-1} - A_{n-1,n}\frac{b_n}{A_{nn}}}{A_{n-1,n-1}}
\end{align}

and so on to row $k$ which yields

\begin{align}
A_{k,k}x_{k} + A_{k,k+1}x_{k+1} +\cdots +  A_{k,n}x_n &= b_{k}\\
\iff A_{k,k}x_{k} + \sum_{j=k+1}^{n}A_{kj}x_j &= b_{k}\\
\implies x_{k} &= \left( b_k - \sum_{j=k+1}^{n}A_{kj}x_j\right)\frac{1}{A_{kk}}
\end{align}

### <span style="color:blue">Exercise 5.5: Back substitution</span>

Extend your code to perform back substitution and hence to obtain the final solution $\boldsymbol{x}$.  Check against the solutions found earlier.  Come up with some random $n\times n$ matrices (you can use `np.random.rand` for that) and check your code against `sl.inv(A)@b` (remember to use the original $A$ and $\boldsymbol{b}$ here of course!)

In [29]:
# This function assumes that A is already an upper triangular matrix, 
# e.g. we have already run our upper_triangular function if needed.

def back_substitution(A, b):
    """ 
    Function to perform back subsitution on the system Ax=b.
    Returns the solution x.
    Assumes that A is on upper triangular form.
    """

    n = np.size(b)
    # Check A is square and its number of rows and columns same as size of the vector b
    rows, cols = np.shape(A)
    assert(rows == cols)
    assert(rows == n)

    # We can/should check that A is upper triangular using np.triu which is the 
    # upper triangular part of a matrix - if A is already upper triangular, then
    # it should of course match the upper-triangular component of A!!

    # np.triu(A) returns upper triangular with the lower bit = 0
    assert(np.allclose(A, np.triu(A)))
    
    x = np.zeros(n)
    # start at the end (row n-1) and work backwards
    for k in range(n-1, -1, -1):
        "first -1 --> stop at 0 (k(n) = 0) second -1 --> go backwards"

        s = 0. # for summation
        for j in range(k+1, n):
            "summation range"
            s = s + A[k, j] * x[j]

        x[k] = (b[k] - s) / A[k, k]

    return x


# This A is the upper triangular matrix carried forward
# from the Python box above, and b the correspondingly updated b vector.
A = np.array([[ 2.,  3., -4.],
              [ 0., -1., 14.],
              [ 0.,  0., 30.]])
"assumes upper triangle in input"

b = np.array([  5., -12., -15.])

# print the solution using our codes
x = back_substitution(A, b)
print('Our solution: ',x) 

# Reinitialise A and b !
# remember our functions overwrote them
# for ans checking
A = np.array([[2., 3., -4.],
                 [6., 8., 2.],
                 [4., 8., -6.]])
b = np.array([5., 3., 19.])


# check our answer against what SciPy gives us by multiplying b by A inverse 
print('SciPy solution: ',sl.inv(A) @ b)

print('Success: ', np.allclose(x, sl.inv(A) @ b))

Our solution:  [-6.   5.  -0.5]
SciPy solution:  [-6.   5.  -0.5]
Success:  True


### Gauss-Jordan elimination

Recall that for the augmented matrix example we did by hand above we continued past the upper-triangular form so that the augmented matrix had the identity matrix in the $A$ location. This algorithm has the name Gauss-Jordan elimination but note that it requires more operations than the conversion to upper-triangular form followed by back subsitution and so is only of academic interest.

### Matrix inversion

Note that if we were to form the augmented equation with the full identity matrix in the place of the vector $\boldsymbol{b}$, i.e. $[A|I]$ and performed row operations exactly as above until $A$ is transformed into the identity matrix $I$, then we would be left with the inverse of $A$ in the original $I$ location, i.e.

$$ [A|I] \rightarrow [I|A^{-1}] $$

### <span style="color:blue">Exercise 5.6: Matrix inversion</span>

Try updating your code to construct the inverse matrix. Check your answer against the inverse matrix given by a built-in function.

Hint: Once you have performed your Gaussian elimination to transform $A$ into an upper triangular matrix, perform another elimination "from bottom to top" to transform $A$ into a diagonal matrix.

In [30]:
# Updated version of the upper_triangular function that
# assumes that a matrix, B, is in the old vector location
# in the augmented system, and applies the same operations to
# B as to A

def upper_triangle2(A, B):
    
    rows_b, cols_b = np.shape(B)
    rows_a, cols_a = np.shape(A)

    "np.shape gives tuple"
    # check A and B are squares (to be invertable)
    assert(rows_a == cols_a)
    assert(rows_b == cols_b)

    # and check A has the same number of rows as rows of B
    assert(rows_a == rows_b)

    # Loop over each pivot row - all but the last row which we will never need to use as a pivot
    for k in range(rows_b - 1):
        # Loop over each row below the pivot row, including the last row which we do need to update
        for i in range(k+1, rows_b):
            "for the row below pivot row"
            "k+1 --> after pivot row. start: k+1 end: n"


            # scaling factor for row
            "Row i (below pivot row) column k"
            "k works bc square matrix"
            s = (A[i, k] / A[k, k])

            # update current row A by looping over column j
            "start from k as we assume entries before this are already zero"
            '''for j in range(k, n):
                A[i, j] = A[i, j] - s * A[k, j]'''
            
            A[i, :] = A[i, :] - s * A[k, :]
            B[i, :] = B[i, :] - s * B[k, :]

                
# Function which transforms the matrix into lower triangular 
# form - the point here is that if you give it a 
# matrix that is already in upper triangular form, then the 
# result will be a diagonal matrix
def lower_triangle2(A, B):
    rows_b, cols_b = np.shape(B)
    rows_a, cols_a = np.shape(A)

    "np.shape gives tuple"
    # check A and B are squares (to be invertable)
    assert(rows_a == cols_a)
    assert(rows_b == cols_b)

    # and check A has the same number of rows as rows of B
    assert(rows_a == rows_b)

    # Loop over each pivot row - all but the last row which we will never need to use as a pivot
    for k in range(rows_b - 1, -1, -1):
        "first -1 --> stop at 0 (k(n) = 0) second -1 --> go backwards"

        # Loop over each row below the pivot row, including the last row which we do need to update
        for i in range(k - 1, -1, -1):
            "for the row below pivot row"
            "k+1 --> after pivot row. start: k+1 end: n"
            # scaling factor for row
            "Row i (below pivot row) column k"
            "k works bc square matrix"
            s = (A[i, k] / A[k, k])
            # update current row A by looping over column j
            "start from k as we assume entries before this are already zero"
            '''for j in range(k, n):
                A[i, j] = A[i, j] - s * A[k, j]'''
            
            A[i, :] = A[i, :] - s * A[k, :]
            B[i, :] = B[i, :] - s * B[k, :]

# Let's redefine A as our matrix above
A = np.array([[2., 3., -4.], 
                [3., -1., 2.], 
                [4., 2., 2.]])

dupe_A = A

# and B is the identity of the corresponding size
" identity matrix becomes inverse"
B = np.eye(np.shape(A)[0])
# takes in row of A

# transform A into upper triangular form (and perform the same operations on B)
upper_triangle2(A, B)
"A carries forward"

print('Upper triangular transformed A = ')
print(A)

# now make this updated A lower triangular as well (the result should be diagonal)
lower_triangle2(A, B)

print('Lower triangular transformed A (carried forward) (= diagonal matrix) ')
print(A)

print('B')
print(B)
# final step is just to divide each row through by the value of the diagonal
# to end up with the identity in the place of A

"eg divide A11 by 2 --> DO THE SAME TO B"
for i in range(np.shape(A)[0]):
    "row_a is no longer stored"
    "DO THE SAME TO B"
    B[i, :] = B[i, :]/A[i, i]
    A[i, :] = A[i, :]/A[i, i]
    "A[i, i] = diagonal value"
    "has to be in this B A order as it modifies Ai afterwards otherwises divides B by 1"

# print final A and B and check solution
print("Final A")
print(A)

print("Final B")
print(B)


# Final check
A = np.array([[2., 3., -4.], 
                [3., -1., 2.], 
                [4., 2., 2.]])

print('Scipy A inverse: ', sl.inv(A))
print('\nSuccess: ', np.allclose(B, sl.inv(A)))

Upper triangular transformed A = 
[[ 2.          3.         -4.        ]
 [ 0.         -5.5         8.        ]
 [ 0.          0.          4.18181818]]
Lower triangular transformed A (carried forward) (= diagonal matrix) 
[[ 2.          0.          0.        ]
 [ 0.         -5.5         0.        ]
 [ 0.          0.          4.18181818]]
B
[[ 0.26086957  0.60869565 -0.08695652]
 [ 0.23913043  2.39130435 -1.91304348]
 [-0.90909091 -0.72727273  1.        ]]
Final A
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [ 0.  0.  1.]]
Final B
[[ 0.13043478  0.30434783 -0.04347826]
 [-0.04347826 -0.43478261  0.34782609]
 [-0.2173913  -0.17391304  0.23913043]]
Scipy A inverse:  [[ 0.13043478  0.30434783 -0.04347826]
 [-0.04347826 -0.43478261  0.34782609]
 [-0.2173913  -0.17391304  0.23913043]]

Success:  True


### <span style="color:blue">Exercise 5.6: Zeros on the diagonal</span>

You may have noticed above that we have no way of guaranteeing that the $A_{kk}$ we divide through by in the Guassian elimination or back substitution algorithms is non-zero (or not very small which will also lead to computational problems).
Note also that we commented that we are free to exchange two rows in our augmented system - how could you use this fact to build robustness into our algorithms in order to deal with matrices for which our algorithms do lead to very small or zero $A_{kk}$ values?  

See if you can figure out how to do this - more on this next week!

In [34]:
# This function swaps rows in matrix A
# (and remember that we need to do likewise for the vector b 
# we are performing the same operations on)

def swap_row(A, b, i, j):
    """ Swap rows i and j of the matrix A and the vector b.
    eg swap row 2 3 --> swap_row(A, b, 1, 2)
    if statement skips func if rows are the same
    """ 
    if i == j:
        return
    print('\nswapping rows', i,'and', j)
    # If we are swapping two values, we need to take a copy of one of them first otherwise
    # we will lose it when we make the first swap and will not be able to use it for the second.
    # We need to make sure it is a real copy - not just a copy of a reference to the data!
    # use np.copy to do this. 
    iA = np.copy(A[i, :])
    ib = np.copy(b[i])

    A[i, :] = A[j, :]
    b[i] = b[j]

    A[j, :] = iA
    b[j] = ib


# This is a new version of the upper_triangular function
# with the added step of swapping rows so the largest
# magnitude number is always our pivot/
# pp stands for partial pivoting which will be explained
# in more detail below.

def upper_trianglepp(A, b):
    """ A function to covert A into upper triangluar form through row operations.
    The same row operations are performed on the vector b.
    
    This version uses partial pivoting.
    
    Note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """
    n = np.size(b)
    # check A is square and its number of rows and columns same as size of the vector b
    rows, cols = np.shape(A)
    assert(rows == cols)
    assert(rows == n)

    # Loop over each pivot row - all but the last row
    for k in range(n-1):
        # Swap rows so we are always dividing through by the largest number.
        # initiatise kmax with the current pivot row (k)
        kmax = k
        # loop over all entries below the pivot and select the k with the largest abs value
        for i in range(k+1, n):
            if abs(A[kmax, k]) < abs(A[i, k]):
                kmax = i
        # and swap the current pivot row (k) with the row with the largest abs value below the pivot
        swap_row(A, b, kmax, k)

        for i in range(k+1, n):
            s = (A[i, k]/A[k, k])
            for j in range(k, n):
                A[i, j] = A[i, j] - s*A[k, j]

            "OUTSIDE the loop"
            b[i] = b[i] - s*b[k]

# Apply the new code with row swaps to our matrix problem from above
A = np.array([[2., 3., -4.],
                 [6., 8., 2.],
                 [4., 8., -6.]])
b = np.array([5., 3., 19.])

orig_A = np.copy(A)
orig_b = np.copy(b)

print('\noriginal A and b')
print(A)
print(b)

upper_trianglepp(A, b)

print('\nupper_trianglepp A b')
print(A)
print(b)

# compute the solution from these using our back substitution code
# could also have used SciPy of course

# A b already modified
x1 = back_substitution(A, b)

print("back_substitutionpp A b")
print(x1)

print("sl.inv(A) @ b")
print(sl.inv(orig_A) @ orig_b)

print('\n Success upper_trianglepp with sl.inv(A) @ b',np.allclose(x1, sl.inv(orig_A) @ orig_b))

# compare with our first function with no row swaps
A = np.array([[2., 3., -4.],
                 [6., 8., 2.],
                 [4., 8., -6.]])
b = np.array([5., 3., 19.])

upper_triangle(orig_A, orig_b)

print('\nA and b without any row swaps (upper_triangle): ')
print(A)
print(b)
x2 = back_substitution(orig_A, orig_b)

print("back_substitution A b no pp")
print(x2)
# check these two systems are equivalent

print('\nThese two upper triangular systems are equivalent (i.e. have the same solution, backsub): ',np.allclose(x1, x2))




original A and b
[[ 2.  3. -4.]
 [ 6.  8.  2.]
 [ 4.  8. -6.]]
[ 5.  3. 19.]

swapping rows 1 and 0

swapping rows 2 and 1

upper_trianglepp A b
[[ 6.00000000e+00  8.00000000e+00  2.00000000e+00]
 [ 0.00000000e+00  2.66666667e+00 -7.33333333e+00]
 [ 0.00000000e+00  5.55111512e-17 -3.75000000e+00]]
[ 3.   13.   -1.25]
back_substitutionpp A b
[-7.33333333  5.79166667  0.33333333]
sl.inv(A) @ b
[-6.   5.  -0.5]

 Success upper_trianglepp with sl.inv(A) @ b False

A and b without any row swaps (upper_triangle): 
[[ 2.  3. -4.]
 [ 6.  8.  2.]
 [ 4.  8. -6.]]
[ 5.  3. 19.]
back_substitution A b no pp
[-6.   5.  -0.5]

These two upper triangular systems are equivalent (i.e. have the same solution, backsub):  False
